In [9]:
import time
from typing import Dict, Any, Hashable

import numpy as np
import networkx as nx
from tqdm import tqdm


def _stationary_distribution(P: np.ndarray,
                             max_iter: int = 10_000,
                             tol: float = 1e-12) -> np.ndarray:
    """Power‑iteration to get the left stationary distribution π of P."""
    n = P.shape[0]
    π = np.full(n, 1.0 / n)
    for _ in range(max_iter):
        π_new = π @ P
        if np.linalg.norm(π_new - π, 1) < tol:
            return π_new
        π = π_new
    return π  # warn: may not have converged


def directed_random_walk_betweenness(
    G: nx.DiGraph,
    assume_strongly_connected: bool = False,
    verbose: bool = True,
) -> Dict[Hashable, float]:
    """Exact random‑walk betweenness for a **directed** graph.

    Parameters
    ----------
    G : nx.DiGraph
        Weighted or unweighted, **strongly‑connected** digraph.
        If dangling nodes exist (out‑degree 0) they are given a uniform
        teleport to keep the chain irreducible.
    assume_strongly_connected : bool, default False
        If *True* skip the connectivity test (saves a few seconds
        on large graphs but may raise if the graph is really not SC).
    verbose : bool, default True
        Print progress and timings.

    Returns
    -------
    dict
        Mapping node → random‑walk betweenness centrality
        (unnormalized).
    """

    t_tot = time.perf_counter()
    n = G.number_of_nodes()
    nodes = list(G.nodes())
    index = {u: i for i, u in enumerate(nodes)}

    # ------------------------------------------------------------------
    # 1) Transition matrix P (row‑stochastic)
    # ------------------------------------------------------------------
    if verbose:
        print(f"[1/5] Building transition matrix  P  ({n}×{n}) …", flush=True)
    P = np.zeros((n, n), dtype=float)

    for u in G:
        i = index[u]
        succ = list(G.successors(u))
        if succ:
            w_total = sum(G[u][v].get("weight", 1.0) for v in succ)
            for v in succ:
                j = index[v]
                P[i, j] = G[u][v].get("weight", 1.0) / w_total
        else:
            # dangling → uniform teleport
            P[i, :] = 1.0 / n

    # ------------------------------------------------------------------
    # 2) Stationary distribution π  (left eigenvector of P)
    # ------------------------------------------------------------------
    if verbose:
        print("[2/5] Power‑iteration for stationary distribution π …",
              flush=True)
    t0 = time.perf_counter()
    π = _stationary_distribution(P)
    if verbose:
        print(f"      converged in {time.perf_counter() - t0:6.2f}s")

    # ------------------------------------------------------------------
    # 3) Random‑walk Laplacian  L_rw  and its pseudoinverse  M
    # ------------------------------------------------------------------
    if verbose:
        print("[3/5] Forming Laplacian  L_rw  and dense pseudoinverse …",
              flush=True)
    t0 = time.perf_counter()
    I = np.eye(n, dtype=float)
    L_rw = np.diag(π) @ (I - P)            # Π · (I − P)
    M = np.linalg.pinv(L_rw, rcond=1e-12)  # Moore–Penrose pseudoinverse
    if verbose:
        print(f"      SVD done in {time.perf_counter() - t0:6.2f}s")

    # ------------------------------------------------------------------
    # 4)   O(n²) accumulation   (vectorised inner loop)
    # ------------------------------------------------------------------
    if verbose:
        print("[4/5] Accumulating betweenness (vectorised)…", flush=True)
    t0 = time.perf_counter()
    diag_M = np.diag(M)
    col_sum = M.sum(axis=0)
    bet = np.zeros(n, dtype=float)

    for j in tqdm(range(n), unit="node"):
        m_jj = diag_M[j]
        csum_j = col_sum[j]

        m_kj = M[:, j]         # column j
        m_jk = M[j, :]         # row j
        m_kk = diag_M          # broadcast

        denom = m_jj - m_kj - m_jk + m_kk
        # avoid /0 when k=j (value unused anyway)
        denom[j] = np.inf

        S = (csum_j - col_sum           # Σ_i m_ij − Σ_i m_ik
             - m_jj + m_jk              #        - m_jj + m_jk
             + (-n + 1) * m_kj          # (-n+1)·m_kj
             + (n - 1) * m_kk)          # (n−1)·m_kk
        S[j] = 0.0  # exclude k=j

        bet[j] = np.divide(S, denom,
                           out=np.zeros_like(S), where=denom != 0).sum()

    if verbose:
        print(f"      done in {time.perf_counter() - t0:6.2f}s")

    # ------------------------------------------------------------------
    # 5) Wrap‑up
    # ------------------------------------------------------------------
    centrality = {nodes[j]: float(bet[j]) for j in range(n)}
    if verbose:
        print(f"[5/5] All done in {time.perf_counter() - t_tot:6.2f}s")

    return centrality


# ----------------------------------------------------------------------
#  Stand‑alone execution
# ----------------------------------------------------------------------
if __name__ == "__main__":
    import argparse, sys, pandas as pd

    parser = argparse.ArgumentParser(
        description="Exact random‑walk betweenness for a directed graph")
    parser.add_argument("--parquet", required=True,
                        help="square adjacency matrix (parquet format)")
    parser.add_argument("--output", default="rw_betweenness.npy",
                        help="output filename (.npy with dict of results)")
    parser.add_argument("--no‑verbose", action="store_true",
                        help="suppress progress messages")

    args = parser.parse_args()

    verbose = not args.no_verbose
    if verbose:
        print("Loading sparsified adjacency matrix …", file=sys.stderr)
    import pandas as pd
    df = pd.read_parquet(args.parquet)
    df.index = df.columns
    G = nx.from_pandas_adjacency(df, create_using=nx.DiGraph())

    res = directed_random_walk_betweenness(G, verbose=verbose)
    np.save(args.output, res)
    if verbose:
        print(f"Saved betweenness dict to  {args.output}")


ModuleNotFoundError: No module named 'tqdm'

In [7]:
import pandas as pd

# 1) load the Parquet
df = pd.read_parquet("dfz_s_2010.parquet")

# 2) melt to edge list of positive weights
rows, cols = df.shape
# df is a square DataFrame: rows=cols=2063
idx = (df > 0).stack()
edges = idx[idx].reset_index()      # only the True entries
edges.columns = ["i", "j", "keep"]
edges["w"] = df.values[ df.values > 0 ]  # grab the positive values
edges = edges.loc[:, ["i","j","w"]]

# 3) save
edges.to_csv("edges.csv", index=False)
print("Wrote edges.csv (shape:", edges.shape, ")")

Wrote edges.csv (shape: (396891, 3) )


In [2]:
import pandas as pd
import networkx as nx
import time

def estimate_betweenness_runtime(G, sample_size=10, weight='weight'):
    """
    Estimate the time required to compute betweenness centrality on a graph G.
    
    The function samples `sample_size` nodes and computes the time needed to
    run a single-source shortest path calculation (as used internally by betweenness centrality)
    and then multiplies the average time by the total number of nodes.
    
    Returns the estimated total time in seconds.
    """
    nodes = list(G.nodes())
    if sample_size > len(nodes):
        sample_size = len(nodes)
    sample_nodes = nodes[:sample_size]
    times = []
    
    print(f"Estimating runtime based on {sample_size} sample nodes:")
    for node in sample_nodes:
        t0 = time.time()
        # Use the weighted single_source_dijkstra_path_length,
        # which is similar to the work done per node in betweenness centrality.
        _ = nx.single_source_dijkstra_path_length(G, node, weight=weight)
        t1 = time.time()
        elapsed = t1 - t0
        times.append(elapsed)
        print(f"  Node {node}: {elapsed:.4f} seconds")
    
    avg_time = sum(times) / len(times)
    estimated_total_time = avg_time * len(nodes)
    
    print(f"Average time per node: {avg_time:.4f} seconds")
    return estimated_total_time

# -------------------------------
# 1. Read the sparsified graph
# -------------------------------
print("Loading sparsified adjacency matrix from dfz_s_2010.parquet ...")
df = pd.read_parquet('dfz_s_2010.parquet')
# Assuming the DataFrame is a square adjacency matrix so that rows match columns
df.index = df.columns  
print(f"Loaded DataFrame of shape {df.shape}")

# Create a directed graph from the DataFrame:
G_directed = nx.from_pandas_adjacency(df, create_using=nx.DiGraph())
print(f"Directed graph has {G_directed.number_of_edges()} edges and {G_directed.number_of_nodes()} nodes.\n")

# -------------------------------
# 2. Estimate the total runtime for betweenness centrality
# -------------------------------
print("Estimating the total betweenness centrality computation time ...")
estimated_time_sec = estimate_betweenness_runtime(G_directed, sample_size=10, weight='weight')
print(f"Estimated total computation time: {estimated_time_sec:.2f} seconds (~ {estimated_time_sec/60:.2f} minutes)\n")

# -------------------------------
# 3. Compute Normal Betweenness Centrality (Directed)
# -------------------------------
print("Starting actual betweenness centrality calculation ...")
start_time = time.time()

bc_directed = nx.betweenness_centrality(G_directed,
                                        normalized=True,
                                        weight='weight',
                                        endpoints=False)
end_time = time.time()
actual_time_sec = end_time - start_time
print(f"Actual betweenness centrality computed in {actual_time_sec:.2f} seconds.\n")

Loading sparsified adjacency matrix from dfz_s_2010.parquet ...
Loaded DataFrame of shape (2063, 2063)
Directed graph has 396891 edges and 2063 nodes.

Estimating the total betweenness centrality computation time ...
Estimating runtime based on 10 sample nodes:
  Node AUS_AGRagr: 0.0705 seconds
  Node AUS_AGRfor: 0.0663 seconds
  Node AUS_AGRfis: 0.0657 seconds
  Node AUS_ENRcoa: 0.0665 seconds
  Node AUS_ENRoil: 0.0667 seconds
  Node AUS_ENRgas: 0.0652 seconds
  Node AUS_MIN+: 0.0645 seconds
  Node AUS_MANfoo: 0.0636 seconds
  Node AUS_MANtex: 0.0649 seconds
  Node AUS_MANwoo: 0.0643 seconds
Average time per node: 0.0658 seconds
Estimated total computation time: 135.75 seconds (~ 2.26 minutes)

Starting actual betweenness centrality calculation ...
Actual betweenness centrality computed in 176.55 seconds.



In [6]:
r_script = """\
# compute_rwbc.R
# --------------

```python
# This Python code will generate an R script for computing
# directed random-walk betweenness (via the tnet package)
# on Google Colab's R runtime.

r_script = """\
# compute_rwbc.R
# ----------------------------------------------
# Install tnet if not already available
if (!requireNamespace("tnet", quietly=TRUE)) {
  install.packages("tnet", repos="https://cloud.r-project.org")
}
library(tnet)

# Read your edge list; adjust filename if different
# Expected format: supplier<TAB>buyer<TAB>weight
edges <- read.csv("supply.tsv", sep="\\t", header=FALSE,
                  col.names=c("i","j","w"))

# Convert to a weighted one‐mode tnet graph
g <- as.tnet(edges, type="weighted one-mode tnet")

# Compute exact random-walk betweenness centrality
# (rw_betweenness_w returns a two-column matrix: node, centrality)
rwbc_mat <- rw_betweenness_w(g)

# Turn into a data frame and save
df <- data.frame(
  node = rwbc_mat[,1],
  rwbc = rwbc_mat[,2]
)
write.csv(df, "rwbc_results.csv", row.names=FALSE)

cat("Done! Results are in rwbc_results.csv\n")
"""

# Write the R script to disk
with open("compute_rwbc.R", "w") as f:
    f.write(r_script)

print("Wrote compute_rwbc.R — upload this (and your supply.tsv) to Colab!")

SyntaxError: unterminated triple-quoted string literal (detected at line 45) (3622392847.py, line 39)

In [4]:
import pandas as pd
import networkx as nx
import time

# -------------------------------
# 1. Read the sparsified graph
# -------------------------------
print("Loading sparsified adjacency matrix from dfz_s_2010.parquet ...")
df = pd.read_parquet('dfz_s_2010.parquet')
df.index = df.columns
print(f"Loaded DataFrame of shape {df.shape}")

# Create a directed graph from the DataFrame:
G_directed = nx.from_pandas_adjacency(df, create_using=nx.DiGraph())
print(f"Directed graph has {G_directed.number_of_edges()} edges and {G_directed.number_of_nodes()} nodes.\n")

# -------------------------------
# 2. Compute Normal Betweenness Centrality (Directed)
# -------------------------------
print("Calculating normal (shortest-path) betweenness centrality for the directed graph ...")
start_time = time.time()
# This function supports directed graphs.
bc_directed = nx.betweenness_centrality(G_directed,
                                        normalized=True,
                                        weight='weight',
                                        endpoints=False)
end_time = time.time()
time_normal = end_time - start_time
print(f"Normal betweenness centrality computed in {time_normal:.2f} seconds.\n")



Loading sparsified adjacency matrix from dfz_s_2010.parquet ...
Loaded DataFrame of shape (2063, 2063)
Directed graph has 396891 edges and 2063 nodes.

Calculating normal (shortest-path) betweenness centrality for the directed graph ...
Normal betweenness centrality computed in 175.80 seconds.



In [ ]:
print("Sample of Normal Betweenness Centrality (directed):")
for i, (node, value) in enumerate(bc_directed.items()):
    if i >= 10:
        break
    print(f"  Node {node}: {value:.5f}")



Sample of Normal Betweenness Centrality (directed):
  Node AUS_AGRagr: 0.00000
  Node AUS_AGRfor: 0.06559
  Node AUS_AGRfis: 0.00341
  Node AUS_ENRcoa: 0.00000
  Node AUS_ENRoil: 0.00000
  Node AUS_ENRgas: 0.00000
  Node AUS_MIN+: 0.00000
  Node AUS_MANfoo: 0.00145
  Node AUS_MANtex: 0.00048
  Node AUS_MANwoo: 0.00145


In [ ]:
# Sort nodes by betweenness centrality in descending order
sorted_bc = sorted(bc_directed.items(), key=lambda item: item[1], reverse=True)

# Print the top 100 nodes
print("Top 100 nodes with the highest betweenness centrality:")
for rank, (node, centrality) in enumerate(sorted_bc[:100], start=1):
    print(f"{rank}. Node: {node}, Betweenness Centrality: {centrality:.5f}")

Top 100 nodes with the highest betweenness centrality:
1. Node: MLT_WATwat, Betweenness Centrality: 0.68652
2. Node: MLT_AGRfor, Betweenness Centrality: 0.59062
3. Node: MLT_MANrep, Betweenness Centrality: 0.30407
4. Node: AUS_TRApos, Betweenness Centrality: 0.28972
5. Node: IDN_AGRfor, Betweenness Centrality: 0.28246
6. Node: SVN_AGRfor, Betweenness Centrality: 0.25463
7. Node: LTU_MANfoo, Betweenness Centrality: 0.24856
8. Node: LUX_AGRfis, Betweenness Centrality: 0.20947
9. Node: RoW_MANcom, Betweenness Centrality: 0.19318
10. Node: CHE_WATwat, Betweenness Centrality: 0.18628
11. Node: CHE_AGRfis, Betweenness Centrality: 0.16272
12. Node: NOR_AGRfor, Betweenness Centrality: 0.15103
13. Node: BGR_AGRagr, Betweenness Centrality: 0.14948
14. Node: IRL_WATwat, Betweenness Centrality: 0.14891
15. Node: SVN_AGRagr, Betweenness Centrality: 0.14449
16. Node: JPN_AGRfor, Betweenness Centrality: 0.13726
17. Node: MLT_MANwoo, Betweenness Centrality: 0.12934
18. Node: IRL_MANtex, Betweenness Ce

In [ ]:
# -------------------------------
# 3. Compute Random Walk (Current-Flow) Betweenness Centrality
# -------------------------------
# NOTE: current_flow_betweenness_centrality does not support DiGraphs.
# As a workaround, we convert the directed graph into an undirected one.
print("Converting the directed graph to undirected for current-flow betweenness centrality computation ...")
G_undirected = G_directed.to_undirected()
print(f"Undirected graph has {G_undirected.number_of_edges()} edges and {G_undirected.number_of_nodes()} nodes.\n")

print("Calculating random walk (current-flow) betweenness centrality (this may be slow ...)")

start_time = time.time()
# You can choose the solver; here we use 'lu' for moderate memory usage.
rw_bc = nx.current_flow_betweenness_centrality(G_undirected,
                                               normalized=True,
                                               weight='weight',
                                               solver='lu')
end_time = time.time()
time_rw = end_time - start_time
print(f"Random walk (current-flow) betweenness centrality computed in {time_rw:.2f} seconds.\n")

# -------------------------------
# 4. Display a Sample of the Results
# -------------------------------
print("\nSample of Random Walk (Current-Flow) Betweenness Centrality (computed on undirected graph):")
for i, (node, value) in enumerate(rw_bc.items()):
    if i >= 10:
        break
    print(f"  Node {node}: {value:.5f}")

print("\nSummary of Computation Times:")
print(f"  Normal Betweenness Centrality: {time_normal:.2f} seconds")
print(f"  Random Walk Betweenness Centrality: {time_rw:.2f} seconds")

# -------------------------------
# 5. Decision Point
# -------------------------------
print("\nBased on the computation times you can decide whether the normal (shortest-path) betweenness centrality is enough or whether the random walk version might provide additional insight at acceptable performance cost.")